In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime

import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

## Glipse Data

Read data

In [ ]:
# btc = pd.read_csv('/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-09-14.csv')

btc = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-12-31.csv')
# Converting the Timestamp column from string to datetime
btc['Timestamp'] = [datetime.fromtimestamp(x) for x in btc['Timestamp']]
btc = btc.set_index('Timestamp')

#### Time resampling

In [ ]:
btc_daily = btc.resample("24H").mean() #daily resampling

#### Time Series Decomposition & Statistical Tests

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
def fill_missing(df):
    ### function to impute missing values using interpolation ###
    df['Open'] = df['Open'].interpolate()
    df['Close'] = df['Close'].interpolate()
    df['Weighted_Price'] = df['Weighted_Price'].interpolate()
    df['Volume_(BTC)'] = df['Volume_(BTC)'].interpolate()
    df['Volume_(Currency)'] = df['Volume_(Currency)'].interpolate()
    df['High'] = df['High'].interpolate()
    df['Low'] = df['Low'].interpolate()
    print(df.head())
    print(df.isnull().sum())
    
fill_missing(btc_daily)

In [ ]:

plt.figure(figsize=(15,12))
series = btc_daily.Weighted_Price
result = seasonal_decompose(series, model='additive',period=1)
result.plot()

In [ ]:
# Renaming the column names accroding to Prophet's requirements

daily_data_fb = btc_daily.reset_index()[['Timestamp','Weighted_Price']].rename({'Timestamp':'ds','Weighted_Price':'y'}, axis=1)
daily_data_fb.head()

In [ ]:
test_length = 365
split_date = "2020-01-01"
train_filt = daily_data_fb['ds'] <= split_date
test_filt = daily_data_fb['ds'] > split_date

train_fb = daily_data_fb[train_filt]
test_fb = daily_data_fb[test_filt]

print("train data shape :", train_fb.shape[0] / daily_data_fb.shape[0] * 100)
print("test data shape :", test_fb.shape[0] / daily_data_fb.shape[0] * 100)

![](https://github.com/AutoViML/Auto_TS/raw/master/logo.png)

[Auto_TS: Auto_TimeSeries](https://github.com/AutoViML/Auto_TS)

In [ ]:
!pip install auto-ts
# !pip install auto_ts --ignore-installed --no-cache-dir 

In [ ]:
!pip install tscv

In [ ]:

from auto_ts import auto_timeseries as ATS


In [ ]:
train_fb = train_fb.reset_index()
train_fb.head()

In [ ]:
test_fb = test_fb.reset_index()
test_fb.head()

In [ ]:

automl_model = ATS(score_type='rmse',
                time_interval='D', forecast_period=30,
                non_seasonal_pdq=None, seasonality=False, seasonal_period=12,
                model_type=['best'],
                verbose=2)

In [ ]:
ts_column = 'ds'
target = 'y'
sep = ','

In [ ]:
automl_model.fit(
    traindata=train_fb,
    ts_column=ts_column,
    target=target,
    cv=5,
    sep=',')

### Evaluate Results

In [ ]:
automl_model.get_leaderboard()

In [ ]:

automl_model.plot_cv_scores()

In [ ]:
# There is a Bug with Prophet folds: https://github.com/AutoViML/Auto_TS/issues/25
automl_model.get_cv_scores()

In [ ]:
results_dict = automl_model.get_ml_dict()
results_dict['Prophet']['forecast'][:5]

In [ ]:
# results_dict['auto_SARIMAX']['forecast'][:5]

#### Forecast Results

In [ ]:
# Using Best Model
future_predictions = automl_model.predict(
    testdata=test_fb
)  
future_predictions

In [ ]:
# Using specific model
future_predictions = automl_model.predict(
    testdata=test_fb,
    model='auto_SARIMAX'
)
future_predictions

In [ ]:
# Using specific model
future_predictions = automl_model.predict(
    testdata=test_fb,
    model='Prophet',
    simple=False

)
future_predictions

In [ ]:
test_fb.head()


In [ ]:
import copy
subm = test_fb[[target]]
print(subm.shape)
subm.head()

In [ ]:
subm['predictions'] = future_predictions['yhat'].values
subm[[target,'predictions']].plot(figsize=(15,8))